# AD9833 Modulators test with FTDI
https://www.analog.com/media/en/technical-documentation/data-sheets/ad9833.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', 'Signal Generators', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline 

from signal_generators import modulators  
from clock_generators.si535x.si5351 import Si5351 
from utilities.adapters import peripherals

Populating the interactive namespace from numpy and matplotlib


## Debug mode?

In [5]:
cls = Si5351

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators and Modulators

In [6]:
with_hardware_device = True

if with_hardware_device:
    _i2c = peripherals.I2C.get_Ftdi_i2c() 
    
else:
    _i2c =  None  # using None for testing without actual hardware device.

si = cls(_i2c)
si2 = cls(_i2c)

In [7]:
bfsk = modulators.BFSK(si)
bpsk = modulators.BPSK(si)
dtmf = modulators.DTMF((si, si2))
fm = modulators.FM(si)
iq = modulators.IQ((si, si2))
ook = modulators.OOK(si)
pm = modulators.PM(si)
pwm = modulators.PWM(si)
qpsk = modulators.QPSK(si)

Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.
Failed in setting multisynth 1 to freq 101000 x 1 = 101000.
Failed in setting multisynth 1 to freq 101000 x 2 = 202000.
Failed in setting multisynth 1 to freq 101000 x 4 = 404000.
Failed in setting multisynth 0 to freq 99000 x 1 = 99000.
Failed in setting multisynth 0 to freq 99000 x 2 = 198000.
Failed in setting multisynth 0 to freq 99000 x 4 = 396000.
Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.
Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.
Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 10000

## Testing data sequence

In [8]:
samples_size = 10
duration = 0.1

symbols = np.random.randint(2, size = samples_size)
digital_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.uniform(-1, 1, size = samples_size).round(5)
analog_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.choice(['1', '2', '3', 'A', '4', '5', '6', 'B', '7', '8', '9', 'C', '*', '0', '#', 'D'], 
                           size = samples_size)
dtmf_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.choice([0, 1, 2, 3], size = samples_size)
quadrature_sequence = [(symbol, duration) for symbol in symbols] 

## Modulators test

In [9]:
def dump(m):
    m._device.dump()

In [10]:
bfsk.send_sequence(digital_sequence)

Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.
Failed in setting multisynth 1 to freq 101000 x 1 = 101000.
Failed in setting multisynth 1 to freq 101000 x 2 = 202000.
Failed in setting multisynth 1 to freq 101000 x 4 = 404000.
Failed in setting multisynth 0 to freq 99000 x 1 = 99000.
Failed in setting multisynth 0 to freq 99000 x 2 = 198000.
Failed in setting multisynth 0 to freq 99000 x 4 = 396000.


AttributeError: 'Si5351' object has no attribute 'select_freq_source'

In [11]:
bpsk.send_sequence(digital_sequence)

Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.


AttributeError: 'Si5351' object has no attribute 'select_phase_source'

In [12]:
ook.send_sequence(digital_sequence)

Failed in setting multisynth 0 to freq 40000 x 1 = 40000.
Failed in setting multisynth 0 to freq 40000 x 2 = 80000.
Failed in setting multisynth 0 to freq 40000 x 4 = 160000.
Failed in setting multisynth 0 to freq 40000 x 8 = 320000.


In [13]:
fm.send_sequence(analog_sequence)

Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.
Failed in setting multisynth 0 to freq 90080.4 x 1 = 90080.4.
Failed in setting multisynth 0 to freq 90080.4 x 2 = 180160.8.
Failed in setting multisynth 0 to freq 90080.4 x 4 = 360321.6.
Failed in setting multisynth 0 to freq 97632.9 x 1 = 97632.9.
Failed in setting multisynth 0 to freq 97632.9 x 2 = 195265.8.
Failed in setting multisynth 0 to freq 97632.9 x 4 = 390531.6.
Failed in setting multisynth 0 to freq 92342.7 x 1 = 92342.7.
Failed in setting multisynth 0 to freq 92342.7 x 2 = 184685.4.
Failed in setting multisynth 0 to freq 92342.7 x 4 = 369370.8.
Failed in setting multisynth 0 to freq 102093.6 x 1 = 102093.6.
Failed in setting multisynth 0 to freq 102093.6 x 2 = 204187.2.
Failed in setting multisynth 0 to freq 102093.6 x 4 = 408374.4.
Failed in setting multisynth 0 to freq 100607.9 x 1 = 100607.9.


In [14]:
-20.1447 % 360

339.8553

In [15]:
pm.send_sequence(analog_sequence)

Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.


In [16]:
qpsk.send_sequence(quadrature_sequence)

Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.


In [17]:
iq.send_sequence(quadrature_sequence)

Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.
Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.
Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.
Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.


AttributeError: 'Si5351' object has no attribute 'select_phase_source'

In [18]:
dtmf.FREQUENCIES = {'row'   : (6970000, 7700000, 8520000, 9410000),
                   'column': (12090000, 13360000, 14770000, 16330000)}

dtmf.TONES = {'1': (dtmf.FREQUENCIES['row'][0], dtmf.FREQUENCIES['column'][0]),
             '2': (dtmf.FREQUENCIES['row'][0], dtmf.FREQUENCIES['column'][1]),
             '3': (dtmf.FREQUENCIES['row'][0], dtmf.FREQUENCIES['column'][2]),
             'A': (dtmf.FREQUENCIES['row'][0], dtmf.FREQUENCIES['column'][3]),
             '4': (dtmf.FREQUENCIES['row'][1], dtmf.FREQUENCIES['column'][0]),
             '5': (dtmf.FREQUENCIES['row'][1], dtmf.FREQUENCIES['column'][1]),
             '6': (dtmf.FREQUENCIES['row'][1], dtmf.FREQUENCIES['column'][2]),
             'B': (dtmf.FREQUENCIES['row'][1], dtmf.FREQUENCIES['column'][3]),
             '7': (dtmf.FREQUENCIES['row'][2], dtmf.FREQUENCIES['column'][0]),
             '8': (dtmf.FREQUENCIES['row'][2], dtmf.FREQUENCIES['column'][1]),
             '9': (dtmf.FREQUENCIES['row'][2], dtmf.FREQUENCIES['column'][2]),
             'C': (dtmf.FREQUENCIES['row'][2], dtmf.FREQUENCIES['column'][3]),
             '*': (dtmf.FREQUENCIES['row'][3], dtmf.FREQUENCIES['column'][0]),
             '0': (dtmf.FREQUENCIES['row'][3], dtmf.FREQUENCIES['column'][1]),
             '#': (dtmf.FREQUENCIES['row'][3], dtmf.FREQUENCIES['column'][2]),
             'D': (dtmf.FREQUENCIES['row'][3], dtmf.FREQUENCIES['column'][3])}
dtmf.SYMBOLS = list(dtmf.TONES.keys())

In [19]:
dtmf.send_sequence(dtmf_sequence)

In [20]:
dtmf._devices[1].clocks[0].set_frequency(25e6)

True

In [21]:
dtmf._devices[1].clocks[0].enable()

In [22]:
dtmf._devices[0].clocks[0].status

OrderedDict([('type', '_Clock'),
             ('idx', 0),
             ('source_freq', 9410000),
             ('my_freq', 9410000),
             ('my_divider', 1),
             ('enabled', False),
             ('oeb_pin_masked', False),
             ('power_downed', False),
             ('phase_offset_enabled', False)])

In [23]:
dtmf._devices[0].clocks[0].freq

9410000

In [ ]:
pwm.run()

Failed in setting multisynth 0 to freq 100000 x 1 = 100000.
Failed in setting multisynth 0 to freq 100000 x 2 = 200000.
Failed in setting multisynth 0 to freq 100000 x 4 = 400000.
